# Detección de fatiga en la conducción de vehículos

## Descripción
Auctorvolumus nostrum iaculis mus torquent ac atqui vehicula scelerisque at nonumy.  Luctusaliquet ponderum nisl mea scelerisque nec graeci.
Torquentac in dicant scripserit oratio regione comprehensam nonumy auctor aliquid conclusionemque delicata periculis fames quem sale iusto euripidis.  Erremvolutpat tempus sed lorem habitasse legere conceptam oporteat similique facilisis.  Nonposse usu erat ea salutatus suspendisse.  Aliquetbrute doctus fastidii moderatius ignota vero mus.  Utamurpurus lacinia ex antiopam ne deserunt.  Comprehensammetus voluptatum praesent egestas consul.


## Objetivo
Auctorvolumus nostrum iaculis mus torquent ac atqui vehicula scelerisque at nonumy.  Luctusaliquet ponderum nisl mea scelerisque nec graeci.
Torquentac in dicant scripserit oratio regione comprehensam nonumy auctor aliquid conclusionemque delicata periculis fames quem sale iusto euripidis.  Erremvolutpat tempus sed lorem habitasse legere conceptam oporteat similique facilisis.  Nonposse usu erat ea salutatus suspendisse.  Aliquetbrute doctus fastidii moderatius ignota vero mus.  Utamurpurus lacinia ex antiopam ne deserunt.  Comprehensammetus voluptatum praesent egestas consul.

In [ ]:
import keras

UNSPLITTED_FOLDER_IMAGES = './datasets/dmd/labels'
SPLITTED_FOLDER_IMAGES = './datasets/dmd/labels_split'

Dividimos las imágenes en train, validación y test en un ratio de:
* 70% train.
* 20% validación.
* 10% test del modelo.

In [ ]:
!pip install split_folders
import splitfolders
import os
if not os.path.exists(SPLITTED_FOLDER_IMAGES):
    splitfolders.ratio(UNSPLITTED_FOLDER_IMAGES, SPLITTED_FOLDER_IMAGES, seed=1, ratio=(.7, .2, .1))

## Análisis exploratorio de datos
Antes de construir el modelo, es fundamental llevar a cabo un proceso de exploración de los datos para identificar y resolver cualquier posible problema. Durante este análisis, verificaremos el equilibrio entre las diferentes clases para evitar cualquier sesgo que pueda afectar a nuestro modelo. Además, nos aseguraremos de que todas las imágenes tengan tres canales y, en caso necesario, normalizaremos los datos.

## Dataset de train, test y validación
Para crear los dataset de train y test, vamos a utilizar la función splitfolders. Como ya hemos descargado algunas fotos de internet para la validación, únicamente dividiremos nuestro dataset original en train y test con una proporción de 80/20.

In [ ]:
INPUT_SHAPE = (224, 224, 3)
TRAIN_DIR = SPLITTED_FOLDER_IMAGES + '/train'
VAL_DIR = SPLITTED_FOLDER_IMAGES + '/val'
TEST_DIR = SPLITTED_FOLDER_IMAGES + '/test'
BATCH_SIZE = 32
IMG_SIZE = (180, 180)
VAL_SPLIT = 0.2

#!pip install chardet
#!pip install --upgrade charset_normalizer
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    labels='inferred',
    label_mode='categorical',
    validation_split=VAL_SPLIT,
    subset='training',
    color_mode='rgb',
    seed=1,
    shuffle=False,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    labels='inferred',
    label_mode='categorical',
    validation_split=VAL_SPLIT,
    subset='validation',
    color_mode='rgb',
    seed=1,
    shuffle=False,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

## Clases equilibradas
Para empezar, verificamos la distribución equilibrada de las clases, lo cual implica que todas ellas contengan la misma cantidad de imágenes. Para llevar a cabo esta comprobación, hemos implementado la función "check_classes", que realiza el análisis y nos proporciona un mensaje indicando si las clases están balanceadas o no, junto con el tamaño de cada una de ellas. Como entrada, solo necesitamos especificar la ruta principal y las diferentes carpetas que contienen las imágenes correspondientes a cada clase:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def showImageGrid(train_ds):

    # Get a batch of images from the dataset
    image_batch, label_batch = next(iter(train_ds))

    # Convert image_batch and label_batch to numpy arrays
    image_batch = image_batch.numpy()
    label_batch = label_batch.numpy()

    # Randomly select 10 images from the batch
    random_indices = np.random.choice(range(len(image_batch)), size=5, replace=False)
    random_images = image_batch[random_indices]
    random_labels = label_batch[random_indices]

    # Create a grid for displaying the images
    fig, axes = plt.subplots(2, 5, figsize=(10, 6))
    axes = axes.flatten()

    # Iterate over the random images and labels
    for i, (image, label) in enumerate(zip(random_images, random_labels)):
        # Display the image
        axes[i].imshow(image.astype(np.uint8))
        axes[i].axis('off')
        axes[i].set_title(np.argmax(label))

    plt.tight_layout()
    plt.show()

showImageGrid(train_ds)

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

Según podemos observar en la salida previa, las clases presentan una distribución equilibrada.

## Número de canales
Además, es crucial verificar que todas las imágenes tengan tres canales para evitar posibles inconvenientes más adelante. Para facilitar esta comprobación, hemos implementado la función "check_channels". Esta función requiere como entrada la ruta principal, los nombres de las subcarpetas y el número de canales que deseamos que tengan las imágenes, en este caso, 3.

In [ ]:
def verifyChannels(train_ds):
    for image_batch, label_batch in train_ds:
        for image in image_batch:
            num_channels = image.shape[-1]  # Get the number of channels
            if num_channels != 3:
                print("Image does not have 3 channels:", image.shape)
        break
    print("All images have 3 channels")

verifyChannels(train_ds)

## Normalización
Por último, en cuanto a la normalización, no sería necesario llevarla a cabo ya que las redes que vamos a utilizar ya contienen una función de preprocesado que importamos directamente desde keras y que realiza todo el tratamiento de los datos para adaptar el input a la red neuronal en cuestión.

### Normalización VGG19
Comenzamos la modelización con la red VGG19 y añadimos las capas necesarias.

In [ ]:
from keras.applications.vgg19 import preprocess_input

#Create VGG19 preprocessing
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y))

# Preprocess the images using ImageDataGenerator
# train_ds = train_ds.map(lambda x, y: (datagen.preprocess_input(x), y))
# val_ds = val_ds.map(lambda x, y: (imageDataGen.preprocess_input(x), y))

showImageGrid(train_ds)
showImageGrid(val_ds)

## Configuración del modelo

In [31]:
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import VGG19

base_model=VGG19(weights='imagenet', include_top=False)

vgg = base_model.output
vgg = GlobalAveragePooling2D()(vgg)
vgg = Dense(512, activation='relu')(vgg)
outputs = Dense(num_classes, activation='softmax')(vgg)

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=IMG_SIZE))

Creamos el modelo y obtenemos el summary

In [32]:
# conda install -c conda-forge python-graphviz
#!pip install keras_visualizer
#!pip install pip install git+https://github.com/raghakot/keras-vis.git -U
from keras.models import Model
# from keras.utils.vis_utils import plot_model
# from keras_visualizer import visualizer

model = Model(inputs=base_model.input, outputs=outputs)
model.summary()
# visualizer(model,file_name='modelVisualization.png', file_format='png', view=True)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0   

In [ ]:
print("Modelo base:", len(base_model.layers), "\nModelo:", len(model.layers))

Indicamos a partir de qué capa el modelo empieza a entrenar. Esto es importante porque si lo entrenamos de 0
nos va a llevar mucho más tiempo. Así , el model sólo aprenderá de las capas que le indiquemos.

In [ ]:
for layer in model.layers[:22]:
    layer.trainable=False
for layer in model.layers[22:]:
    layer.trainable=True
model.summary()

Vemos como el número de parámetros entrenables es de 20.288.579, correspondientes a las capas que no hemos congelado y que vamos a entrenar a continuación. Para ello, creamos el generador de train y test usando como función de preproceso la de la red VGG19, fijamos los directorios donde se encuentran los dataset e indicamos como tamaño objetivo 224x224:

Compilamos el modelo utilizando como optimizador "Adam" y como función de coste la entropía cruzada (Cross-entropy). Lo entrenaremos para 50 épocas pero activamos los callbacks de 'early_stopping' y 'best_model_checkpoint':

In [23]:
model.compile(optimizer='Adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
from keras.models import load_model

MODEL_SAVE_FOLDER = './models/cnn/cnn_vgg/'

# if os.path.exists(MODEL_SAVE_FOLDER):
modelvgg19 =  load_model(MODEL_SAVE_FOLDER +'best_cnn_vgg_model_saturday')
modelvgg19.hi
# else:
#     early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
#     best_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(MODEL_SAVE_FOLDER +'best_cnn_vgg_model_saturday', monitor='loss', save_best_only=True, mode='min')
#     modelvgg19 = model.fit(
#         x=train_ds,
#         validation_data = val_ds,
#         epochs=10,
#         callbacks=[early_stop,best_model_checkpoint])

Una vez que el modelo ha sido entrenado, procedemos a visualizar gráficamente la precisión y la función de coste tanto para el conjunto de entrenamiento como para el conjunto de prueba. Esta representación gráfica nos permitirá analizar el rendimiento del modelo en ambas situaciones y evaluar su capacidad de generalización.

In [26]:
import matplotlib.pyplot as plt

acc      = modelvgg19.history['accuracy']
val_acc  = modelvgg19.history['val_accuracy']
loss     = modelvgg19.history['loss']
val_loss = modelvgg19.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Entrenamiento acc')
plt.plot(epochs, val_acc, 'b', label='Validación acc')
plt.title('Accuracy - exactitud de entrenamiento y validación')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Entrenamiento loss')
plt.plot(epochs, val_loss, 'b', label='Validación loss')
plt.title('Loss - función objetivo en entrenamiento y prueba')
plt.legend()

plt.show()

TypeError: 'NoneType' object is not subscriptable

Ponderumquot utinam periculis facilis errem quam tortor sanctus prompta mucius wisi posse feugait quo ne invenire.  Mineglegentur his deterruisset docendi sadipscing tristique arcu in.  Natumex ocurreret saepe nibh viderer.  Ultricesvocibus donec minim.

## Predicciones
Después de haber seleccionado el modelo, nos adentramos en la etapa de hacer predicciones utilizando las imágenes que hemos obtenido de Internet y que están almacenadas en la carpeta "testeo". Para llevar a cabo estas predicciones, hemos desarrollado una función que requiere como entrada la ruta al directorio de imágenes, la función de preprocesamiento elegida y el modelo seleccionado. Esta función nos devolverá las imágenes junto con sus respectivas predicciones: